In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from flask import Flask
import threading
import time
import json
import io
import base64
import matplotlib.pyplot as plt

# Initialize Flask app
app = Flask(__name__)

# Initialize DataFrame
df = pd.DataFrame(columns=["Crypto Name", "Price", "Timestamp"], dtype=str)

def scrape_coin(url, coin_name):
    """Scrape a single coin's data given its URL and name."""
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Scrape the price
    # Keeping the same logic and classes as requested
    crypto_price = soup.find('span', class_='sc-65e7f566-0', attrs={'data-test': 'text-cdp-price-display'})
    final_price = crypto_price.text.replace('$', '').strip() if crypto_price else "Price not found"

    # Prepare the new row
    date_time = datetime.now()
    new_row = {"Crypto Name": coin_name, "Price": final_price, "Timestamp": str(date_time)}
    return new_row

def scrape_data():
    global df
    # Bitcoin
    bitcoin_url = 'https://coinmarketcap.com/currencies/bitcoin/'
    bitcoin_name = 'Bitcoin'
    bitcoin_row = scrape_coin(bitcoin_url, bitcoin_name)

    # Algorand
    algo_url = 'https://coinmarketcap.com/currencies/algorand/'
    algo_name = 'Algorand'
    algo_row = scrape_coin(algo_url, algo_name)

    # Concatenate the new rows
    df = pd.concat([df, pd.DataFrame([bitcoin_row]), pd.DataFrame([algo_row])], ignore_index=True)
    print("Data scraped:", bitcoin_row)
    print("Data scraped:", algo_row)

    # Update the global df
    globals()['df'] = df

@app.route('/')
def render_pandas_table():
    global df
    if df.empty:
        return "<h1>No data available yet.</h1>"

    # Make a copy of df to avoid modifying the original
    temp_df = df.copy()
    temp_df['Timestamp'] = pd.to_datetime(temp_df['Timestamp'])
    # Remove commas from Price and convert to float
    temp_df['Price'] = temp_df['Price'].str.replace(',', '', regex=False)
    temp_df['Price'] = pd.to_numeric(temp_df['Price'], errors='coerce')

    # ----- MINUTE-BASED LOGIC -----
    # If you want to compare by minute of the hour:
    temp_df['TimeUnit'] = temp_df['Timestamp'].dt.minute
    x_label = 'Minute of the Hour'
    chart_title_btc = 'Bitcoin Price by Minute'
    chart_title_algo = 'Algorand Price by Minute'
    sleep_interval = 60  # 1 minute

    # ----- HOUR-BASED LOGIC  -----
    # If you want to compare by hour of the day:
    # temp_df['TimeUnit'] = temp_df['Timestamp'].dt.hour
    # x_label = 'Hour of the Day'
    # chart_title_btc = 'Bitcoin Price by Hour'
    # chart_title_algo = 'Algorand Price by Hour'
    # sleep_interval = 3600  # 1 hour

    # Sort by timestamp to ensure line plot continuity
    temp_df = temp_df.sort_values(by='Timestamp')

    # Separate the data by coin
    temp_df_btc = temp_df[temp_df['Crypto Name'] == 'Bitcoin']
    temp_df_algo = temp_df[temp_df['Crypto Name'] == 'Algorand']

    # Generate Bitcoin chart
    fig_btc, ax_btc = plt.subplots(figsize=(4, 3))
    ax_btc.plot(temp_df_btc['TimeUnit'], temp_df_btc['Price'], color='blue', marker='o', linestyle='-', linewidth=2)
    ax_btc.set_title(chart_title_btc)
    ax_btc.set_xlabel(x_label)
    ax_btc.set_ylabel('Price (USD)')

    buf_btc = io.BytesIO()
    plt.tight_layout()
    fig_btc.savefig(buf_btc, format='png')
    plt.close(fig_btc)
    buf_btc.seek(0)
    plot_data_btc = base64.b64encode(buf_btc.read()).decode('utf-8')

    # Generate Algorand chart
    fig_algo, ax_algo = plt.subplots(figsize=(4, 3))
    ax_algo.plot(temp_df_algo['TimeUnit'], temp_df_algo['Price'], color='green', marker='o', linestyle='-', linewidth=2)
    ax_algo.set_title(chart_title_algo)
    ax_algo.set_xlabel(x_label)
    ax_algo.set_ylabel('Price (USD)')

    buf_algo = io.BytesIO()
    plt.tight_layout()
    fig_algo.savefig(buf_algo, format='png')
    plt.close(fig_algo)
    buf_algo.seek(0)
    plot_data_algo = base64.b64encode(buf_algo.read()).decode('utf-8')

    # Create the HTML for the tables
    # One table for Bitcoin data
    btc_table_html = (df[df['Crypto Name'] == 'Bitcoin']
                      .to_html(index=False, border=1))

    # One table for Algorand data
    algo_table_html = (df[df['Crypto Name'] == 'Algorand']
                       .to_html(index=False, border=1))

    # Create HTML layout with two charts side by side and two tables side by side
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Comparative Crypto Price Trends</title>
        <style>
            .container {{
                display: flex;
                justify-content: space-around;
                align-items: flex-start;
                margin-bottom: 20px;
            }}
            .tables {{
                display: flex;
                justify-content: space-around;
                align-items: flex-start;
            }}
            .chart-container, .table-container {{
                margin: 10px;
            }}
        </style>
    </head>
    <body>
        <h1>Comparative Crypto Price Trends</h1>
        <div class="container">
            <div class="chart-container">
                <img src="data:image/png;base64,{plot_data_btc}" alt="Bitcoin Chart">
            </div>
            <div class="chart-container">
                <img src="data:image/png;base64,{plot_data_algo}" alt="Algorand Chart">
            </div>
        </div>
        <div class="tables">
            <div class="table-container">
                <h2>Bitcoin Data</h2>
                {btc_table_html}
            </div>
            <div class="table-container">
                <h2>Algorand Data</h2>
                {algo_table_html}
            </div>
        </div>
    </body>
    </html>
    """
    return html

@app.route('/data')
def data_route():
    global df
    data = df.to_dict(orient='records')
    json_data = json.dumps(data, indent=2)  # Pretty-print with indent
    return app.response_class(response=json_data, status=200, mimetype='application/json')

def run_scraper():
    # Adjust this interval based on whether you chose hour or minute logic above.
    # For minute-based comparison:
    sleep_interval = 60
    # For hour-based comparison, uncomment:
    # sleep_interval = 3600
    
    while True:
        scrape_data()
        time.sleep(sleep_interval)

def run_flask():
    app.run(debug=False, use_reloader=False, host='0.0.0.0', port=5000)

# Start threads
scraper_thread = threading.Thread(target=run_scraper, daemon=True)
flask_thread = threading.Thread(target=run_flask, daemon=True)
scraper_thread.start()
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.140:5000
Press CTRL+C to quit


Data scraped: {'Crypto Name': 'Bitcoin', 'Price': '99,985.64', 'Timestamp': '2024-12-08 11:05:21.725740'}
Data scraped: {'Crypto Name': 'Algorand', 'Price': '0.4955', 'Timestamp': '2024-12-08 11:05:21.895671'}


127.0.0.1 - - [08/Dec/2024 11:05:23] "GET / HTTP/1.1" 200 -


Data scraped: {'Crypto Name': 'Bitcoin', 'Price': '99,985.64', 'Timestamp': '2024-12-08 11:06:22.141195'}
Data scraped: {'Crypto Name': 'Algorand', 'Price': '0.4955', 'Timestamp': '2024-12-08 11:06:22.349202'}
Data scraped: {'Crypto Name': 'Bitcoin', 'Price': '100,039.78', 'Timestamp': '2024-12-08 11:07:22.588011'}
Data scraped: {'Crypto Name': 'Algorand', 'Price': '0.4955', 'Timestamp': '2024-12-08 11:07:22.785554'}
Data scraped: {'Crypto Name': 'Bitcoin', 'Price': '100,039.78', 'Timestamp': '2024-12-08 11:08:23.097148'}
Data scraped: {'Crypto Name': 'Algorand', 'Price': '0.4955', 'Timestamp': '2024-12-08 11:08:23.313022'}
